# Loading and splitting Dataset
The goal of this notebook is to continue the development of the DeepTropism model using a Pytorch.<br>
The dataset was already created on the previous notebook and all the HIV-1 env V3 loop sequences where aligned.


In [24]:
# Libraries used on the analysis
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import os
import random

First we load the Dataframe with all the sequences published on referenced articles.<br>
The D


In [3]:
df = pd.read_csv('/home/gabriel/Documents/Repos/DeepTropism/dataset_unique_seqs_aligned_gapopen_15_old.tsv', 
                 sep='\t', names=['name', 'dataset', 'label','sequence'])
df.head()

,name,dataset,label,sequence
0,CCR5_1471_29187_CN_2003_B,geno2pheno,CCR5,CTQTQQQY-KKKY---T----------SRTR-ASM-----V-CNRR...
1,CCR5/CXCR4/CCR1/CCR2b/CCR3/CCR4_MVP5180_67_CM_...,geno2pheno,R5X4,CIREGIAE-VQDI---Y--T--G-P-----M-RWRSMTLKR-SNNT...
2,RKF859742,newdb,CCR5,CERPTMDI-QDIH------I--G-P-----M-AWYSTYIER-QAKG...
3,RAF009608,hivcopred,CCR5,CSRPEMDV-QEIR---N-----G-P-----M-AWYSMALAK-GGTT...
4,RKF859743,newdb,CCR5,CRRPAMKV-QEMR---I----------G--PMAWY-----S-MALE...


In [4]:
df = df[df.label != 'validation']
df.shape

(3572, 4)

In [5]:
# Function to call labels
def tropism_label(row):
    # For CCR5
    if row.label == 'CCR5':
        return 0
    # For CXCR4
    elif row.label == 'CXCR4':
        return 1
    # For R5X4
    elif row.label == 'R5X4':
        return 1

In [6]:
df['label_numeric'] = df.apply(tropism_label, axis=1)

In [7]:
df.label.value_counts()

CCR5     2757
R5X4      484
CXCR4     331
Name: label, dtype: int64

In [8]:
df.label_numeric.value_counts()

0    2757
1     815
Name: label_numeric, dtype: int64

# Spliting the Dataset for Cross Validation
We are going to create indices and set it to variables to make our cross validation reproducible. Our dataset is going to consist on:<br>
* Trainning = 80 %
* Validation = 10 %
* Test = 10 %

In [9]:
# Create a list of indices and shuffle it using seed
random.seed(23)
size = df.shape[0]
list_indices = list(range(size))
random.shuffle(list_indices)

In [10]:
# Now create the list of indices for trainning, validation and test
test_indices = list_indices[:int(size/10)]
train_val_indices = list_indices[int(size/10):]

In [11]:
len(test_indices)

357

In [12]:
len(train_val_indices)

3215

In [13]:
assert len(train_val_indices) + len(test_indices) == len(list_indices), "Splitting indices with error"

## Creating the Dataloaders for trainning

In [14]:
def get_array_from_sequence(protein_sequence):
    """
    Function to convert a protein sequence into a tensor.
    Each amino acid is represented by an numpy array of zeros of size 26,
    and the dict_aa_pos defines the position to be converted to 1.
    
    The function iterates over the protein sequences and stacks the arrays.
    At the end the arrays are linearized and converted to a tensor of size
    n x 26, with n the size of the protein.
    
    If the character is not present on the dict_aa_pos (eg. '-') the respective
    array is formed by zeros, and represents a missing value.
    """
    dict_aa_pos = {
    'A':1, 'R':2, 'N':3, 'D':4, 'C':5, 'Q':6, 'E':7, 'G':8,
    'H':9, 'I':10, 'L':11, 'K':12, 'M':13, 'F':14, 'P':15, 
    'O':16, 'S':17, 'U':18, 'T':19, 'W':20, 'Y':21, 'V':22, 
    'B':23, 'Z':24, 'J':25, 'X':0}
    
    f_array = np.zeros(26)
    for aa in protein_sequence:
        arr = np.zeros(26)
        if dict_aa_pos.get(aa):
            arr[dict_aa_pos.get(aa)] = 1
        f_array = np.vstack((f_array, arr))
    f_array = np.delete(f_array, 0,0)
    
    #return torch.from_numpy((f_array.flatten()).astype(float))
    return f_array.flatten().astype(float)

In [15]:
# Create list to append data from the df
list_data = []
list_labels = []

# Convert the sequences and labels to arrays to use as data on pytorch
for index, row in df.iterrows():
    list_data.append(get_array_from_sequence(str(row.sequence)))
    list_labels.append(int(row.label_numeric))

In [16]:
len(list_data) == len(list_labels)

True

In [18]:
# For Test set
test_data = []
test_label = []
for j in test_indices:
    test_data.append(list_data[j])
    test_label.append(np.array(list_labels[j]))

test_tensor_x = torch.stack([torch.from_numpy(i) for i in test_data]) # transform to torch tensors
test_tensor_y = torch.stack([torch.from_numpy(i) for i in test_label])

test_dataset = torch.utils.data.TensorDataset(test_tensor_x,test_tensor_y) # create your test dataset
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=64) # create your dataloader


In [28]:
# Now we define the
len(train_val_indices)

3215

In [59]:
crossval_val_indices = np.array_split(train_val_indices,5)

In [60]:
len(crossval_val_indices[0])

643

In [62]:
# For Trainning and validation set
# Define the cross validation indices for trainning and validation sets
crossval_val_indices = np.array_split(train_val_indices,5)

# Iterate over crossval_val_indices defining the Dataloaders
for n in range(len(crossval_val_indices)):
    print(f'Cross Validation: {n + 1}')
    trainning_data = []
    trainning_label = []
    validation_data = []
    validation_label = []

    validation_indices = list(crossval_val_indices[n])
    trainning_indices = list(set(train_val_indices) - set(validation_indices))

    for j in validation_indices:
        validation_data.append(list_data[j])
        validation_label.append(np.array(list_labels[j]))

    validation_tensor_x = torch.stack([torch.from_numpy(i) for i in validation_data]) # transform to torch tensors
    validation_tensor_y = torch.stack([torch.from_numpy(i) for i in validation_label])

    validation_dataset = torch.utils.data.TensorDataset(validation_tensor_x,validation_tensor_y) # create your test dataset
    validation_dataloader = torch.utils.data.DataLoader(validation_dataset, batch_size=64) # create your dataloader
    
    for k in trainning_indices:
        trainning_data.append(list_data[k])
        trainning_label.append(np.array(list_labels[k]))

    trainning_tensor_x = torch.stack([torch.from_numpy(i) for i in trainning_data]) # transform to torch tensors
    trainning_tensor_y = torch.stack([torch.from_numpy(i) for i in trainning_label])

    trainning_dataset = torch.utils.data.TensorDataset(trainning_tensor_x,trainning_tensor_y) # create your test dataset
    trainning_dataloader = torch.utils.data.DataLoader(trainning_dataset, batch_size=64) # create your dataloader

Cross Validation: 1
Cross Validation: 2
Cross Validation: 3
Cross Validation: 4
Cross Validation: 5


## Define the Deep Neural Network Architecture


In [ ]:
class DeepTropism_1(nn.Module):
    def __init__(self):
        super(DeepTropism_1, self).__init__()
        self.linear1 = nn.Linear(1560,250)
        self.linear2 = nn.Linear(250,100)
        self.linear3 = nn.Linear(100,2)
    
    def forward(self,X):
        X = F.relu(self.linear1(X))
        X = F.relu(self.linear2(X))
        X = self.linear3(X)
        return F.log_softmax(X, dim=1)
 
model = DeepTropism_1().float()
#model = model.float()
model

In [95]:
# Define the Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.003, momentum=0.9)

In [97]:
# For Trainning and validation set
# Define the cross validation indices for trainning and validation sets
crossval_val_indices = np.array_split(train_val_indices,5)

# Iterate over crossval_val_indices defining the Dataloaders
for n in range(len(crossval_val_indices)):
    print(f'Cross Validation: {n + 1}')
    trainning_data = []
    trainning_label = []
    validation_data = []
    validation_label = []

    validation_indices = list(crossval_val_indices[n])
    trainning_indices = list(set(train_val_indices) - set(validation_indices))

    for j in validation_indices:
        validation_data.append(list_data[j])
        validation_label.append(np.array(list_labels[j]))

    validation_tensor_x = torch.stack([torch.from_numpy(i) for i in validation_data]) # transform to torch tensors
    validation_tensor_y = torch.stack([torch.from_numpy(i) for i in validation_label])

    validation_dataset = torch.utils.data.TensorDataset(validation_tensor_x,validation_tensor_y) # create your test dataset
    validation_dataloader = torch.utils.data.DataLoader(validation_dataset, batch_size=64) # create your dataloader
    
    for k in trainning_indices:
        trainning_data.append(list_data[k])
        trainning_label.append(np.array(list_labels[k]))

    trainning_tensor_x = torch.stack([torch.from_numpy(i) for i in trainning_data]) # transform to torch tensors
    trainning_tensor_y = torch.stack([torch.from_numpy(i) for i in trainning_label])

    trainning_dataset = torch.utils.data.TensorDataset(trainning_tensor_x,trainning_tensor_y) # create your test dataset
    trainning_dataloader = torch.utils.data.DataLoader(trainning_dataset, batch_size=64) # create your dataloader
    
    # Instantiante new model
    #model = DeepTropism_1().float()
        
    # Define Cross Validation Trainning Loop
    for epoch in range(600):  # loop over the dataset multiple times

        running_loss = 0.0
        for i, data in enumerate(trainning_dataloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs.float())
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            #print(running_loss)
            #if i % 3239 == 0:    # print every 3239 mini-batches
            if epoch % 20 == 0 and i % 3239 == 0:
                print('[%d, %3d] loss: %.9f' %
                      (epoch + 1, i + 1, running_loss / 50))
            running_loss = 0.0

    print('Finished Training')

    correct = 0
    total = 0
    error = 0
    labels_array = np.empty([0])
    predict_array = np.empty([0])

    with torch.no_grad():
        for data in validation_dataloader:
            images, labels = data
            outputs = model(images.float())
            _, predicted = torch.max(outputs.data, 1)

            labels_array = np.concatenate([labels_array, labels])
            predict_array = np.concatenate([predict_array, predicted])

            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            error += (predicted != labels).sum().item()

    print(f'Neural Network accuracy Cross Validation {n}: {round(100.0 * correct/total, 2)}%')
    
    torch.save(model.state_dict(), f'model_cv{n+1}.ptb')

Cross Validation: 1
[1,         1] loss: 0.00004
[21,         1] loss: 0.00001
[41,         1] loss: 0.00000
[61,         1] loss: 0.00000
[81,         1] loss: 0.00000
[101,         1] loss: 0.00000
[121,         1] loss: 0.00000
[141,         1] loss: 0.00000
[161,         1] loss: 0.00000
[181,         1] loss: 0.00000
[201,         1] loss: 0.00000
[221,         1] loss: 0.00000
[241,         1] loss: 0.00000
[261,         1] loss: 0.00000
[281,         1] loss: 0.00000
[301,         1] loss: 0.00000
[321,         1] loss: 0.00000
[341,         1] loss: 0.00000
[361,         1] loss: 0.00000
[381,         1] loss: 0.00000
[401,         1] loss: 0.00000
[421,         1] loss: 0.00000
[441,         1] loss: 0.00000
[461,         1] loss: 0.00000
[481,         1] loss: 0.00000
[501,         1] loss: 0.00000
[521,         1] loss: 0.00000
[541,         1] loss: 0.00000
[561,         1] loss: 0.00000
[581,         1] loss: 0.00000
Finished Training
Neural Network accuracy Cross Validati

In [99]:
correct = 0
total = 0
error = 0
labels_array = np.empty([0])
predict_array = np.empty([0])

with torch.no_grad():
    for data in test_dataloader:
        images, labels = data
        outputs = model(images.float())
        _, predicted = torch.max(outputs.data, 1)
        
        labels_array = np.concatenate([labels_array, labels])
        predict_array = np.concatenate([predict_array, predicted])
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        error += (predicted != labels).sum().item()

print(f'Neural Network accuracy: {round(100.0 * correct/total, 2)}%')

Neural Network accuracy: 91.04%


In [100]:
def show_metrics(y_true, y_score):
    # True positive
    tp = np.sum(y_true * y_score)
    # False positive
    fp = np.sum((y_true == 0) * y_score)
    # True negative
    tn = np.sum((y_true==0) * (y_score==0))
    # False negative
    fn = np.sum(y_true * (y_score==0))

    # True positive rate (sensitivity or recall)
    tpr = tp / (tp + fn)
    # False positive rate (fall-out)
    fpr = fp / (fp + tn)
    # Precision
    precision = tp / (tp + fp)
    # True negatvie tate (specificity)
    tnr = 1 - fpr
    # F1 score
    f1 = 2*tp / (2*tp + fp + fn)
    # ROC-AUC for binary classification
    auc = (tpr+tnr) / 2
    # MCC
    mcc = (tp * tn - fp * fn) / np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))

    print("True positive: ", tp)
    print("False positive: ", fp)
    print("True negative: ", tn)
    print("False negative: ", fn)

    print("True positive rate (recall): ", tpr)
    print("False positive rate: ", fpr)
    print("Precision: ", precision)
    print("True negative rate: ", tnr)
    print("F1: ", f1)
    print("ROC-AUC: ", auc)
    print("MCC: ", mcc)

In [101]:
show_metrics(labels_array, predict_array)

True positive:  62.0
False positive:  12.0
True negative:  263
False negative:  20.0
True positive rate (recall):  0.7560975609756098
False positive rate:  0.04363636363636364
Precision:  0.8378378378378378
True negative rate:  0.9563636363636363
F1:  0.7948717948717948
ROC-AUC:  0.856230598669623
MCC:  0.7393080105538697
